# Análisis y Limpieza de Datos - Tabla Clientes

Este notebook contiene el análisis completo de la tabla **Clientes** para identificar problemas de calidad de datos, realizar limpieza, estandarización y normalización.

## Objetivos:
1. **Análisis de problemas**: Identificar duplicados, valores nulos, formatos incorrectos
2. **Limpieza**: Eliminar espacios extra, corregir formatos
3. **Estandarización**: Unificar formatos de fechas, emails y texto
4. **Normalización**: Verificar relaciones con otras tablas

## Herramientas utilizadas:
- **Pandas**: Para manipulación y análisis de datos
- **Numpy**: Para operaciones numéricas eficientes

In [17]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
import os
from datetime import datetime
import re

# Configuración para mostrar más columnas y filas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("Librerías importadas correctamente ✓")

Librerías importadas correctamente ✓


## 1. Carga de Datos

Cargamos la tabla **Clientes** y las tablas relacionadas para entender la estructura completa de los datos.

In [18]:
# Definir rutas de archivos
ruta_base = '../Base_de_datos/'
ruta_limpia = '../Base_de_datos_limpia/'

# Crear carpeta de destino si no existe
os.makedirs(ruta_limpia, exist_ok=True)

# Cargar tabla principal Clientes
df_clientes = pd.read_excel(ruta_base + 'Clientes.xlsx')

# Cargar tablas relacionales para verificar integridad
df_ventas = pd.read_excel(ruta_base + 'Ventas.xlsx')
df_productos = pd.read_excel(ruta_base + 'Productos.xlsx')
df_detalle_ventas = pd.read_excel(ruta_base + 'Detalle_ventas.xlsx')

print("✓ Datos cargados correctamente")
print(f"Clientes: {df_clientes.shape[0]} filas, {df_clientes.shape[1]} columnas")
print(f"Ventas: {df_ventas.shape[0]} filas")
print(f"Productos: {df_productos.shape[0]} filas")
print(f"Detalle Ventas: {df_detalle_ventas.shape[0]} filas")

✓ Datos cargados correctamente
Clientes: 100 filas, 5 columnas
Ventas: 120 filas
Productos: 100 filas
Detalle Ventas: 343 filas


## 2. Exploración Inicial

Examinamos la estructura y características básicas de la tabla Clientes.

In [19]:
# Exploración inicial de la tabla Clientes
print("=== INFORMACIÓN GENERAL ===")
print(f"Dimensiones: {df_clientes.shape}")
print(f"Columnas: {list(df_clientes.columns)}")
print("\n=== TIPOS DE DATOS ===")
print(df_clientes.dtypes)
print("\n=== PRIMERAS 3 FILAS ===")
print(df_clientes.head(3))
print("\n=== INFORMACIÓN ESTADÍSTICA ===")
print(df_clientes.describe(include='all'))

=== INFORMACIÓN GENERAL ===
Dimensiones: (100, 5)
Columnas: ['id_cliente', 'nombre_cliente', 'email', 'ciudad', 'fecha_alta']

=== TIPOS DE DATOS ===
id_cliente                 int64
nombre_cliente            object
email                     object
ciudad                    object
fecha_alta        datetime64[ns]
dtype: object

=== PRIMERAS 3 FILAS ===
   id_cliente   nombre_cliente                     email      ciudad  \
0           1    Mariana Lopez    mariana.lopez@mail.com  Carlos Paz   
1           2    Nicolas Rojas    nicolas.rojas@mail.com  Carlos Paz   
2           3  Hernan Martinez  hernan.martinez@mail.com  Rio Cuarto   

  fecha_alta  
0 2023-01-01  
1 2023-01-02  
2 2023-01-03  

=== INFORMACIÓN ESTADÍSTICA ===
        id_cliente nombre_cliente                   email      ciudad  \
count   100.000000            100                     100         100   
unique         NaN             95                     100           6   
top            NaN   Olivia Perez  mariana.l

## 3. Análisis de Problemas

Identificamos problemas específicos en cada columna: duplicados, valores nulos, formatos incorrectos.

In [20]:
# Análisis detallado de problemas en los datos
print("=== ANÁLISIS DE PROBLEMAS ===\n")

# 1. Verificar valores nulos en cada columna
print("1. VALORES NULOS POR COLUMNA:")
valores_nulos = df_clientes.isnull().sum()
print(valores_nulos[valores_nulos > 0])  # Solo mostrar columnas con nulos
if valores_nulos.sum() == 0:
    print("✓ No hay valores nulos")

# 2. Verificar duplicados en ID_cliente
print("\n2. ANÁLISIS DE ID_CLIENTE:")
duplicados_id = df_clientes['id_cliente'].duplicated().sum()
print(f"IDs duplicados: {duplicados_id}")
if duplicados_id > 0:
    print("IDs duplicados encontrados:")
    print(df_clientes[df_clientes['id_cliente'].duplicated(keep=False)]['id_cliente'].values)

# 3. Verificar filas completamente duplicadas
print("\n3. FILAS DUPLICADAS COMPLETAS:")
filas_duplicadas = df_clientes.duplicated().sum()
print(f"Filas duplicadas: {filas_duplicadas}")

# 4. Verificar problemas en nombres (espacios extra, caracteres especiales)
print("\n4. PROBLEMAS EN NOMBRE_CLIENTE:")
nombres_con_espacios = df_clientes['nombre_cliente'].str.strip() != df_clientes['nombre_cliente']
print(f"Nombres con espacios al inicio/final: {nombres_con_espacios.sum()}")

# 5. Verificar problemas en emails
print("\n5. PROBLEMAS EN EMAIL:")
# Patrón básico para validar emails
patron_email = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
emails_invalidos = ~df_clientes['email'].str.match(patron_email, na=False)
print(f"Emails con formato inválido: {emails_invalidos.sum()}")
if emails_invalidos.sum() > 0:
    print("Ejemplos de emails inválidos:")
    print(df_clientes.loc[emails_invalidos, 'email'].head(3).values)

=== ANÁLISIS DE PROBLEMAS ===

1. VALORES NULOS POR COLUMNA:
Series([], dtype: int64)
✓ No hay valores nulos

2. ANÁLISIS DE ID_CLIENTE:
IDs duplicados: 0

3. FILAS DUPLICADAS COMPLETAS:
Filas duplicadas: 0

4. PROBLEMAS EN NOMBRE_CLIENTE:
Nombres con espacios al inicio/final: 0

5. PROBLEMAS EN EMAIL:
Emails con formato inválido: 0


In [21]:
# Continuación del análisis de problemas - Fechas y ciudades
print("6. PROBLEMAS EN FECHA_ALTA:")
# Verificar formato de fechas
try:
    # Intentar convertir fechas para detectar problemas
    fechas_convertidas = pd.to_datetime(df_clientes['fecha_alta'], errors='coerce')
    fechas_invalidas = fechas_convertidas.isnull().sum()
    print(f"Fechas con formato inválido: {fechas_invalidas}")
    
    # Verificar fechas futuras (después de hoy)
    fecha_actual = datetime.now()
    fechas_futuras = (fechas_convertidas > fecha_actual).sum()
    print(f"Fechas futuras (después de hoy): {fechas_futuras}")
    
except Exception as e:
    print(f"Error al analizar fechas: {e}")

print("\n7. PROBLEMAS EN CIUDAD:")
ciudades_con_espacios = df_clientes['ciudad'].str.strip() != df_clientes['ciudad']
print(f"Ciudades con espacios al inicio/final: {ciudades_con_espacios.sum()}")

# Verificar ciudades con caracteres especiales o números
ciudades_problematicas = df_clientes['ciudad'].str.contains(r'[0-9!@#$%^&*()]', na=False)
print(f"Ciudades con caracteres especiales/números: {ciudades_problematicas.sum()}")

print("\n8. RESUMEN DE PROBLEMAS ENCONTRADOS:")
total_problemas = (
    duplicados_id + filas_duplicadas + nombres_con_espacios.sum() + 
    emails_invalidos.sum() + ciudades_con_espacios.sum() + ciudades_problematicas.sum()
)
print(f"Total de registros con problemas: {total_problemas}")
print(f"Porcentaje de datos con problemas: {(total_problemas/len(df_clientes)*100):.2f}%")

6. PROBLEMAS EN FECHA_ALTA:
Fechas con formato inválido: 0
Fechas futuras (después de hoy): 0

7. PROBLEMAS EN CIUDAD:
Ciudades con espacios al inicio/final: 0
Ciudades con caracteres especiales/números: 0

8. RESUMEN DE PROBLEMAS ENCONTRADOS:
Total de registros con problemas: 0
Porcentaje de datos con problemas: 0.00%


## 4. Verificación de Integridad Relacional

Verificamos que los IDs de clientes en la tabla **Clientes** coincidan con los de las tablas relacionadas (**Ventas**).

In [22]:
# Verificación de integridad relacional
print("=== VERIFICACIÓN DE INTEGRIDAD RELACIONAL ===\n")

# Obtener IDs únicos de cada tabla
ids_clientes = set(df_clientes['id_cliente'].unique())
ids_ventas = set(df_ventas['id_cliente'].unique()) if 'id_cliente' in df_ventas.columns else set()

print(f"IDs únicos en Clientes: {len(ids_clientes)}")
print(f"IDs únicos en Ventas: {len(ids_ventas)}")

# Verificar clientes sin ventas (huérfanos)
if ids_ventas:
    clientes_sin_ventas = ids_clientes - ids_ventas
    print(f"Clientes sin ventas: {len(clientes_sin_ventas)}")
    
    # Verificar ventas de clientes inexistentes
    ventas_sin_cliente = ids_ventas - ids_clientes
    print(f"Ventas con clientes inexistentes: {len(ventas_sin_cliente)}")
    
    if ventas_sin_cliente:
        print(f"IDs problemáticos en ventas: {list(ventas_sin_cliente)[:5]}...")  # Mostrar solo 5
else:
    print("No se puede verificar integridad - columna id_cliente no encontrada en Ventas")

# Verificar emails duplicados (un problema común)
print(f"\n=== EMAILS DUPLICADOS ===")
emails_duplicados = df_clientes['email'].duplicated().sum()
print(f"Emails duplicados: {emails_duplicados}")
if emails_duplicados > 0:
    print("Emails que aparecen múltiples veces:")
    emails_repetidos = df_clientes[df_clientes['email'].duplicated(keep=False)]
    print(emails_repetidos[['id_cliente', 'nombre_cliente', 'email']].head())

=== VERIFICACIÓN DE INTEGRIDAD RELACIONAL ===

IDs únicos en Clientes: 100
IDs únicos en Ventas: 67
Clientes sin ventas: 33
Ventas con clientes inexistentes: 0

=== EMAILS DUPLICADOS ===
Emails duplicados: 0


## 5. Limpieza y Estandarización

Aplicamos correcciones a los problemas identificados usando operaciones eficientes de **pandas**.

In [23]:
# Crear copia para limpieza (preservar datos originales)
df_clientes_limpio = df_clientes.copy()

print("=== PROCESO DE LIMPIEZA ===\n")

# 1. Limpiar espacios en blanco en todas las columnas de texto
print("1. Eliminando espacios extra...")
columnas_texto = df_clientes_limpio.select_dtypes(include=['object']).columns
for col in columnas_texto:
    # Usar str.strip() para eliminar espacios al inicio y final
    df_clientes_limpio[col] = df_clientes_limpio[col].astype(str).str.strip()

# 2. Estandarizar formato de nombres (primera letra mayúscula)
print("2. Estandarizando nombres...")
df_clientes_limpio['nombre_cliente'] = df_clientes_limpio['nombre_cliente'].str.title()

# 3. Estandarizar ciudades (primera letra mayúscula)
print("3. Estandarizando ciudades...")
df_clientes_limpio['ciudad'] = df_clientes_limpio['ciudad'].str.title()

# 4. Convertir emails a minúsculas (estándar)
print("4. Estandarizando emails...")
df_clientes_limpio['email'] = df_clientes_limpio['email'].str.lower()

# 5. Estandarizar formato de fechas
print("5. Estandarizando fechas...")
df_clientes_limpio['fecha_alta'] = pd.to_datetime(df_clientes_limpio['fecha_alta'], errors='coerce')

# 6. Eliminar filas duplicadas completas si existen
filas_antes = len(df_clientes_limpio)
df_clientes_limpio = df_clientes_limpio.drop_duplicates()
filas_despues = len(df_clientes_limpio)
print(f"6. Filas duplicadas eliminadas: {filas_antes - filas_despues}")

print(f"\n✓ Limpieza completada. Registros procesados: {len(df_clientes_limpio)}")

=== PROCESO DE LIMPIEZA ===

1. Eliminando espacios extra...
2. Estandarizando nombres...
3. Estandarizando ciudades...
4. Estandarizando emails...
5. Estandarizando fechas...
6. Filas duplicadas eliminadas: 0

✓ Limpieza completada. Registros procesados: 100


## 6. Validación de Datos Limpios

Verificamos que las correcciones se aplicaron correctamente y identificamos registros que requieren atención manual.

In [24]:
# Validación post-limpieza
print("=== VALIDACIÓN POST-LIMPIEZA ===\n")

# 1. Verificar emails después de la limpieza
patron_email = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
emails_invalidos_post = ~df_clientes_limpio['email'].str.match(patron_email, na=False)
print(f"1. Emails inválidos restantes: {emails_invalidos_post.sum()}")

# 2. Verificar fechas nulas después de conversión
fechas_nulas = df_clientes_limpio['fecha_alta'].isnull().sum()
print(f"2. Fechas que no se pudieron convertir: {fechas_nulas}")

# 3. Verificar IDs duplicados
ids_duplicados_post = df_clientes_limpio['id_cliente'].duplicated().sum()
print(f"3. IDs duplicados restantes: {ids_duplicados_post}")

# 4. Identificar registros problemáticos que necesitan revisión manual
print(f"\n=== REGISTROS QUE REQUIEREN ATENCIÓN MANUAL ===")

# Crear un DataFrame con todos los problemas
problemas = pd.DataFrame({
    'id_cliente': df_clientes_limpio['id_cliente'],
    'email_invalido': emails_invalidos_post,
    'fecha_nula': df_clientes_limpio['fecha_alta'].isnull(),
    'id_duplicado': df_clientes_limpio['id_cliente'].duplicated(keep=False)
})

# Filtrar registros con al menos un problema
registros_problematicos = problemas[
    problemas['email_invalido'] | 
    problemas['fecha_nula'] | 
    problemas['id_duplicado']
]

print(f"Total de registros con problemas: {len(registros_problematicos)}")

if len(registros_problematicos) > 0:
    print("\nPrimeros 5 registros problemáticos:")
    ids_problematicos = registros_problematicos['id_cliente'].head(5)
    print(df_clientes_limpio[df_clientes_limpio['id_cliente'].isin(ids_problematicos)])
else:
    print("✓ Todos los registros están limpios")

=== VALIDACIÓN POST-LIMPIEZA ===

1. Emails inválidos restantes: 0
2. Fechas que no se pudieron convertir: 0
3. IDs duplicados restantes: 0

=== REGISTROS QUE REQUIEREN ATENCIÓN MANUAL ===
Total de registros con problemas: 0
✓ Todos los registros están limpios


## 7. Normalización Final

Aplicamos las últimas optimizaciones y creamos el dataset final normalizado.

In [25]:
# Normalización final y optimizations
print("=== NORMALIZACIÓN FINAL ===\n")

# 1. Ordenar por ID para consistencia
df_clientes_final = df_clientes_limpio.sort_values('id_cliente').reset_index(drop=True)

# 2. Optimizar tipos de datos para eficiencia
print("1. Optimizando tipos de datos...")

# Convertir ID a entero si es posible
try:
    df_clientes_final['id_cliente'] = pd.to_numeric(df_clientes_final['id_cliente'], downcast='integer')
except:
    print("   - ID_cliente mantiene formato original")

# 3. Crear columnas adicionales útiles para análisis
print("2. Creando columnas derivadas...")

# Calcular antigüedad del cliente (días desde registro)
fecha_actual = datetime.now()
df_clientes_final['dias_como_cliente'] = (fecha_actual - df_clientes_final['fecha_alta']).dt.days

print("✓ Normalización completada")
print(f"✓ Dataset final: {len(df_clientes_final)} registros")
print(f"✓ Columnas finales: {list(df_clientes_final.columns)}")

=== NORMALIZACIÓN FINAL ===

1. Optimizando tipos de datos...
2. Creando columnas derivadas...
✓ Normalización completada
✓ Dataset final: 100 registros
✓ Columnas finales: ['id_cliente', 'nombre_cliente', 'email', 'ciudad', 'fecha_alta', 'dias_como_cliente']


## 8. Reporte Final y Guardado

Generamos un reporte de todas las transformaciones realizadas y guardamos los datos limpios.

In [26]:
# Reporte final de transformaciones
print("=== REPORTE FINAL DE LIMPIEZA ===\n")

# Comparación antes vs después
print("RESUMEN DE TRANSFORMACIONES:")
print(f"📊 Registros originales: {len(df_clientes)}")
print(f"📊 Registros finales: {len(df_clientes_final)}")
print(f"📊 Registros eliminados: {len(df_clientes) - len(df_clientes_final)}")

print(f"\n📋 CAMBIOS APLICADOS:")
print("   ✓ Eliminación de espacios extra en textos")
print("   ✓ Estandarización de nombres (formato título)")
print("   ✓ Estandarización de ciudades (formato título)")
print("   ✓ Conversión de emails a minúsculas")
print("   ✓ Estandarización de formato de fechas")
print("   ✓ Eliminación de filas duplicadas")
print("   ✓ Optimización de tipos de datos")
print("   ✓ Creación de columnas derivadas")

print(f"\n📈 CALIDAD DE DATOS:")
emails_validos = df_clientes_final['email'].str.match(patron_email, na=False).sum()
fechas_validas = df_clientes_final['fecha_alta'].notna().sum()
print(f"   ✓ Emails válidos: {emails_validos}/{len(df_clientes_final)} ({emails_validos/len(df_clientes_final)*100:.1f}%)")
print(f"   ✓ Fechas válidas: {fechas_validas}/{len(df_clientes_final)} ({fechas_validas/len(df_clientes_final)*100:.1f}%)")
print(f"   ✓ IDs únicos: {df_clientes_final['id_cliente'].nunique() == len(df_clientes_final)}")

# Mostrar muestra de datos finales
print(f"\n📋 MUESTRA DE DATOS LIMPIOS:")
print(df_clientes_final.head(3))

=== REPORTE FINAL DE LIMPIEZA ===

RESUMEN DE TRANSFORMACIONES:
📊 Registros originales: 100
📊 Registros finales: 100
📊 Registros eliminados: 0

📋 CAMBIOS APLICADOS:
   ✓ Eliminación de espacios extra en textos
   ✓ Estandarización de nombres (formato título)
   ✓ Estandarización de ciudades (formato título)
   ✓ Conversión de emails a minúsculas
   ✓ Estandarización de formato de fechas
   ✓ Eliminación de filas duplicadas
   ✓ Optimización de tipos de datos
   ✓ Creación de columnas derivadas

📈 CALIDAD DE DATOS:
   ✓ Emails válidos: 100/100 (100.0%)
   ✓ Fechas válidas: 100/100 (100.0%)
   ✓ IDs únicos: True

📋 MUESTRA DE DATOS LIMPIOS:
   id_cliente   nombre_cliente                     email      ciudad  \
0           1    Mariana Lopez    mariana.lopez@mail.com  Carlos Paz   
1           2    Nicolas Rojas    nicolas.rojas@mail.com  Carlos Paz   
2           3  Hernan Martinez  hernan.martinez@mail.com  Rio Cuarto   

  fecha_alta  dias_como_cliente  
0 2023-01-01               102

In [27]:
# Guardar datos limpios en la carpeta destino
archivo_salida = ruta_limpia + 'Clientes_limpio.xlsx'

# Guardar en Excel
df_clientes_final.to_excel(archivo_salida, index=False)

# Guardar también en CSV para compatibilidad
archivo_csv = ruta_limpia + 'Clientes_limpio.csv'
df_clientes_final.to_csv(archivo_csv, index=False, encoding='utf-8')

print("=== ARCHIVOS GUARDADOS ===")
print(f"✅ Excel: {archivo_salida}")
print(f"✅ CSV: {archivo_csv}")

# Crear reporte de limpieza
reporte = f"""
REPORTE DE LIMPIEZA - TABLA CLIENTES
=====================================
Fecha de procesamiento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

RESUMEN:
- Registros originales: {len(df_clientes)}
- Registros finales: {len(df_clientes_final)}
- Registros eliminados: {len(df_clientes) - len(df_clientes_final)}

TRANSFORMACIONES APLICADAS:
1. Eliminación de espacios extra
2. Estandarización de nombres y ciudades
3. Normalización de emails
4. Conversión de fechas
5. Eliminación de duplicados
6. Optimización de tipos de datos
7. Creación de columnas derivadas

CALIDAD FINAL:
- Emails válidos: {emails_validos}/{len(df_clientes_final)}
- Fechas válidas: {fechas_validas}/{len(df_clientes_final)}
- IDs únicos: {df_clientes_final['id_cliente'].nunique() == len(df_clientes_final)}
"""

# Guardar reporte
with open(ruta_limpia + 'Reporte_Limpieza_Clientes.txt', 'w', encoding='utf-8') as f:
    f.write(reporte)

print(f"✅ Reporte: {ruta_limpia}Reporte_Limpieza_Clientes.txt")
print(f"\n🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!")
print(f"📁 Archivos disponibles en: {ruta_limpia}")

=== ARCHIVOS GUARDADOS ===
✅ Excel: ../Base_de_datos_limpia/Clientes_limpio.xlsx
✅ CSV: ../Base_de_datos_limpia/Clientes_limpio.csv
✅ Reporte: ../Base_de_datos_limpia/Reporte_Limpieza_Clientes.txt

🎉 ¡PROCESO COMPLETADO EXITOSAMENTE!
📁 Archivos disponibles en: ../Base_de_datos_limpia/


## 🎯 Conclusiones

### ✅ **Proceso Completado**
El análisis y limpieza de la tabla **Clientes** se completó exitosamente. Los datos están ahora:

- **Limpios**: Sin espacios extra ni formatos inconsistentes
- **Estandarizados**: Formatos uniformes en nombres, ciudades y emails  
- **Normalizados**: Tipos de datos optimizados y columnas derivadas añadidas
- **Validados**: Verificación de integridad relacional con otras tablas

### 📊 **Archivos Generados**
Los resultados están guardados en la carpeta `Base_de_datos_limpia/`:
- `Clientes_limpio.xlsx` - Datos limpios en formato Excel
- `Clientes_limpio.csv` - Datos limpios en formato CSV
- `Reporte_Limpieza_Clientes.txt` - Reporte detallado del proceso

### 🔄 **Próximos Pasos**
Los datos limpios de **Clientes** están listos para:
- Análisis de ventas y patrones de compra
- Integración con las tablas **Ventas**, **Productos** y **Detalle_ventas**
- Creación de dashboards y reportes

In [29]:
#Muestra de los datos limpios cargados desde el archivo guardado

Clientes = '../Base_de_datos_limpia/Clientes_limpio.xlsx'

c = pd.read_excel(Clientes, index_col='id_cliente')
c

,nombre_cliente,email,ciudad,fecha_alta,dias_como_cliente
id_cliente,,,,,
1,Mariana Lopez,mariana.lopez@mail.com,Carlos Paz,2023-01-01,1027
2,Nicolas Rojas,nicolas.rojas@mail.com,Carlos Paz,2023-01-02,1026
3,Hernan Martinez,hernan.martinez@mail.com,Rio Cuarto,2023-01-03,1025
4,Uma Martinez,uma.martinez@mail.com,Carlos Paz,2023-01-04,1024
5,Agustina Flores,agustina.flores@mail.com,Cordoba,2023-01-05,1023
6,Uma Medina,uma.medina@mail.com,Villa Maria,2023-01-06,1022
7,Emilia Castro,emilia.castro@mail.com,Rio Cuarto,2023-01-07,1021
8,Bruno Castro,bruno.castro@mail.com,Carlos Paz,2023-01-08,1020
9,Yamila Molina,yamila.molina@mail.com,Carlos Paz,2023-01-09,1019
